In [2]:
import pandas as pd
from pyproj import Proj
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set(color_codes=True)
import collections

%matplotlib inline

In [10]:
data_path = "data/merged/manhattan_brooklyn_2014_2015.csv"
df = pd.DataFrame.from_csv(data_path, index_col = False)
df.head()

,borough,block,schooldist,council,zipcode,ltdheight,splitzone,bldgclass,landuse,easements,...,sale_price,sale_date,tax_class_at_time_of_sale,year_built,residential_units,commercial_units,total_units,bbl_pluto,unit_bbl,price_per_sqft
0,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,1598000,2014-04-30,2,1920,1,0,1,3.000208e+09,3.000201e+09,57.515117
1,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,1598000,2014-04-30,2,1920,1,0,1,3.000208e+09,3.000201e+09,57.515117
2,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,3309312,2014-05-23,2,1920,1,0,1,3.000208e+09,3.000201e+09,119.108552
3,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,3309312,2014-05-23,2,1920,1,0,1,3.000208e+09,3.000201e+09,119.108552
4,BK,20.0,13.0,33.0,11201.0,0.0,0.0,RM,4.0,1.0,...,1210000,2014-06-09,2,1920,1,0,1,3.000208e+09,3.000201e+09,43.550245


In [12]:
df.shape

(84948, 51)

In [11]:
no_dups = df.drop_duplicates()
no_dups.shape

(20089, 51)

In [13]:
data_path = "data/merged/finance_manhattan_brooklyn_2014_2015.csv"
finance = pd.DataFrame.from_csv(data_path, index_col = False)
finance.head()

,sale_price,sale_date,tax_class_at_time_of_sale,year_built,residential_units,commercial_units,total_units,block,bbl
0,0,2014-03-21,4,1920,0,90,90,1,3000010050
1,1598000,2014-04-30,2,1920,1,0,1,20,3000201001
2,3309312,2014-05-23,2,1920,1,0,1,20,3000201002
3,1210000,2014-06-09,2,1920,1,0,1,20,3000201003
4,1333907,2014-05-21,2,1920,1,0,1,20,3000201004


In [14]:
finance.shape

(138672, 9)

In [15]:
finance_no_dups = finance.drop_duplicates()
finance_no_dups.shape

(69336, 9)

In [17]:
data_path = "data/merged/pluto_manhattan_brooklyn_2014_2015.csv"
pluto = pd.DataFrame.from_csv(data_path, index_col = False)
pluto.head()

,borough,block,schooldist,council,zipcode,ltdheight,splitzone,bldgclass,landuse,easements,...,condono,xcoord,ycoord,zonemap,latitude,longitude,gross_sqft_pluto,garage,extension,countalter
0,BK,1,13.0,33.0,11201.0,0,0.0,D7,4.0,0,...,0.0,987624.0,195992.0,12d,40.704629,-73.987831,123199.0,0,0,1
1,BK,1,13.0,33.0,11201.0,0,1.0,E9,6.0,0,...,0.0,987838.0,195989.0,12d,40.704621,-73.987059,154400.0,0,0,2
2,BK,7,13.0,33.0,11201.0,0,0.0,G1,10.0,0,...,0.0,987216.0,195803.0,12d,40.704110,-73.989303,9585.0,0,0,1
3,BK,16,13.0,33.0,11201.0,0,0.0,Q0,9.0,0,...,0.0,986856.0,195975.0,12d,40.704582,-73.990601,5000.0,0,0,0
4,BK,18,13.0,33.0,11201.0,0,0.0,D5,3.0,0,...,0.0,987577.0,195811.0,12d,40.704132,-73.988001,211386.0,0,0,1


In [18]:
pluto_no_dups = pluto.drop_duplicates()
pluto_no_dups.shape

(305187, 41)

In [19]:
pluto.shape

(305187, 41)

In [35]:
finance.groupby(['sale_date'])

In [43]:
data_path = "data/merged/bronx_brooklyn_manhattan_queens_statenisland_2003_2016.csv"
df = pd.DataFrame.from_csv(data_path, index_col = False)
df.head()

,zipcode_x,ltdheight,splitzone,easements,comarea,resarea,numbldgs,numfloors,unitsres,unitstotal,...,lottype_1.0,lottype_2.0,lottype_3.0,lottype_4.0,lottype_5.0,lottype_6.0,lottype_7.0,tax_class_at_time_of_sale_1,tax_class_at_time_of_sale_2,tax_class_at_time_of_sale_3
0,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [44]:
df.shape

(187896, 205)

In [49]:
df[['sidewalk_cafes_dist','subwaydist']].isnull().sum()

sidewalk_cafes_dist    4105
subwaydist             4105
dtype: int64

In [50]:
def fill_na(data_train, data_test):
    '''
    Fills NaN values with the mean of the column. Note we have already created dummy variables
    for columns with missing values.
    
    Args:
        data_train: Pandas dataframe used for training.
        data_test: Pandas dataframe used for testing.
    Returns:
        data_train: Pandas dataframe with no NaN values, ready for modeling.
        data_test: Pandas dataframe with no NaN values, ready for testing.
    
    '''
    data_train = data_train.apply(lambda x: x.fillna(x.mean()),axis=0)
    data_test = data_test.apply(lambda x: x.fillna(x.mean()),axis=0)
    return data_train, data_test

In [51]:
def split_data(data):
    '''
    Splits data into training and test sets (0.8/0.2)
        Args: 
            data: Pandas dataframe
        Returns:
            data_train: Pandas dataframe used for training
            data_test: Pandas dataframe used for testing
    
    '''
    #Convert 'int64' into float; otherwise, sklearn throws a warning message
    columns = data.columns.values
    non_float = []
    for col in columns:
        if data[col].dtype != np.float64:
            non_float.append(col)
    for col in non_float:
        data[col] = data[col].astype(float)
    #drop NaN for crucial columns
    data= data.dropna(how = 'any', subset = ['price_per_sqft'])   
    #Split the data
    split = cross_validation.ShuffleSplit(data.shape[0], n_iter=1, train_size = 0.8, test_size=.2, random_state = 1)

    for train, test in split:
        train_index = train
        test_index = test
    data_train = data.ix[train_index,:]
    data_test = data.ix[test_index,:]
    data_train.reset_index(drop=True, inplace=True)
    data_test.reset_index(drop=True, inplace=True)
    return data_train, data_test

In [55]:
df = df.drop(['zonemap','sale_date','sale_price','year_built','latitude','longitude','bbl','bbl_x','bbl_y'], axis=1)
data_train, data_test = split_data(df)

In [59]:
data_train, data_test = fill_na(data_train, data_test)
for col in data_train.columns.values:
    try:
        df[col] = df[col].astype('float32')
    except:
        print(col)

In [62]:
for col in data_train.columns.values:
    if np.any(np.isnan(data_train[col])):
        print(col)

public_recycling_bins_dist


In [65]:
df['public_recycling_bins_dist'].unique()

array([ nan])

In [19]:
data_path = "data/merged/bronx_brooklyn_manhattan_queens_statenisland_2003_2016.csv"
new_df = pd.DataFrame.from_csv(data_path, index_col = False)
new_df.head()

,zipcode,ltdheight,splitzone,easements,comarea,resarea,numbldgs,numfloors,unitsres,unitstotal,...,lottype_1.0,lottype_2.0,lottype_3.0,lottype_4.0,lottype_5.0,lottype_6.0,lottype_7.0,tax_class_at_time_of_sale_1,tax_class_at_time_of_sale_2,tax_class_at_time_of_sale_3
0,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [30]:
new_df.columns.values

array(['zipcode', 'ltdheight', 'splitzone', 'easements', 'comarea',
       'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'lotfront', 'lotdepth', 'bldgfront', 'bldgdepth', 'irrlotcode',
       'bsmtcode', 'yearbuilt', 'builtcode', 'histdist', 'landmark', 'bbl',
       'xcoord', 'ycoord', 'zonemap', 'latitude', 'longitude',
       'gross_sqft_pluto', 'garage', 'extension', 'countalter',
       'sale_price', 'sale_date', 'year_built', 'residential_units',
       'commercial_units', 'total_units', 'price_per_sqft', 'subwaydist',
       'botanical_gardens_dist', 'path_stations_dist', 'libraries_dist',
       'colleges_dist', 'sidewalk_cafes_dist', 'day_care_centers_dist',
       'dsny_special_waste_dist', 'link_nyc_centers_dist',
       'beacon_centers_dist', 'out_of_school_youth_centers_dist',
       'runaway_homeless_youth_centers_dist',
       'agency_service_centers_dist',
       'summer_youth_employment_centers_dist',
       'adult_continuing_ed_centers_dist',
  

In [38]:
new_df.shape

(187896, 195)

In [ ]:
new_df.i

In [39]:
df_no_dups = new_df.drop_duplicates(subset = ['bbl','latitude','longitude'])
df_no_dups.shape

(17648, 195)

In [40]:
df_no_dups[['bbl','latitude','longitude','zipcode']].to_csv("bbl_lat_long_zip.csv", index = False)

In [14]:
new_df['bldgclass'].unique()

array(['Y', 'R', 'S', 'K', 'D', 'O', 'C', 'H', 'B', 'A', 'G', 'L', 'I',
       'E', 'W', 'J', 'P', 'Z', 'M', 'N', 'Z9', 'V', 'F', 'Q'], dtype=object)

In [49]:
data_path = "data/merged/manhattan_2010_2010.csv"
pluto = pd.DataFrame.from_csv(data_path, index_col = False)
pluto.head()

,zipcode,ltdheight,splitzone,easements,comarea,resarea,numbldgs,numfloors,unitsres,unitstotal,...,proxcode_2.0,lottype_0.0,lottype_1.0,lottype_2.0,lottype_3.0,lottype_4.0,lottype_5.0,lottype_6.0,tax_class_at_time_of_sale_1,tax_class_at_time_of_sale_2
0,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10004.0,0.0,0.0,0.0,1016406.0,0.0,1.0,22.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10004.0,0.0,0.0,0.0,15218.0,4260.0,1.0,6.0,3.0,4.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10004.0,0.0,0.0,0.0,10000.0,168748.0,1.0,45.0,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,10004.0,0.0,0.0,0.0,10000.0,168748.0,1.0,45.0,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:
pluto.shape

(8730, 161)

In [52]:
pluto.columns.values

array(['zipcode', 'ltdheight', 'splitzone', 'easements', 'comarea',
       'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'lotfront', 'lotdepth', 'bldgfront', 'bldgdepth', 'irrlotcode',
       'bsmtcode', 'yearbuilt', 'builtcode', 'histdist', 'landmark', 'bbl',
       'xcoord', 'ycoord', 'zonemap', 'latitude', 'longitude',
       'gross_sqft_pluto', 'garage', 'extension', 'countalter',
       'sale_price', 'sale_date', 'year_built', 'residential_units',
       'commercial_units', 'total_units', 'price_per_sqft', 'subwaydist',
       'botanical_gardens_dist', 'path_stations_dist', 'libraries_dist',
       'colleges_dist', 'sidewalk_cafes_dist', 'day_care_centers_dist',
       'dsny_special_waste_dist', 'link_nyc_centers_dist',
       'beacon_centers_dist', 'out_of_school_youth_centers_dist',
       'runaway_homeless_youth_centers_dist',
       'agency_service_centers_dist',
       'summer_youth_employment_centers_dist',
       'adult_continuing_ed_centers_dist',
  